# 01. Preprocesado de Datos Vectoriales

Este notebook tiene como objetivo principal la preparación y el filtrado de datos vectoriales para su posterior uso en el proyecto. Se inicia con el preprocesamiento de los marcos de las imágenes PNOA IRG, obteniendo el ámbito de estudio del proyecto. A continuación, se aborda la integración y el filtrado de datos de la Base Topográfica Nacional (BTN) del CNIG, centrándose en las capas relacionadas con superficies de agua como embalses, lagunas y balsas. Se realiza un filtrado espacial utilizando la información de la Red de Información Ambiental de Andalucía (REDIAM) para refinar la selección de balsas de riego y/o ganaderas. Finalmente, se aplica un filtrado por superficie mínima y se eliminan manualmente algunas geometrías consideradas no relevantes mediante una inspección visual. El resultado de este proceso son conjuntos de datos vectoriales más 'limpios' y mejor preparados preparados para etapas posteriores del proyecto.


**Objetivo**: Preparar los datos vectoriale en crudo (raw) para su uso en el proyecto.

**Contenido**:

* Carga de los datos.
* Unión de datos.
* Delimitación del ámbito de estudio.
* Guardado de datos preprocesados en GeoPackage.
* Realización de comprobaciones varias.

Antes de utilizarlos datos vectoriales que necesitamos, debemos realizar algunas transformaciones previas para adaptarlos a nuestro ámbito de estudio. Estos tratamientos incluyen:
* Conversión de formatos: GeoPackage (.gpkg) es un formato ampliamente utilizado para datos vectoriales, por lo que realizaremos conversiones si es necesario.
* Recorte por el ámbito de estudio: Limitaremos los datos vectoriales a la extensión geográfica definida para nuestro proyecto.
* Otras transformaciones: Se aplicarán otras transformaciones según sea necesario para preparar los datos.

## Importamos librerias necesarias

In [1]:
# Instalar las librerias necesarias
# !pip install pyarrow

In [2]:
# Importar librerias necesarias
import pandas as pd
import geopandas as gpd
import glob
import os
import folium
import zipfile
import tempfile
from shapely.ops import unary_union
from rtree import index

## Marcos de las imágenes PNOA IRG (ámbito de estudio)
**Web**: https://centrodedescargas.cnig.es/CentroDescargas/ortofoto-pnoa-falso-color-infrarrojo

En la web del Centro de descargas del CNIG, los archivos de las imágenes **PNOA IRG** vienen acompañados de un archivo comprimido .zip que contiene el un archivo .shp con el marco de cada una de las imágenes individuales de las hojas NTN25. Por comodidad los tenemos descargados junto con las imágenes en la ruta ../data/raw/raster/PNOA_IRG/

* Recorreremos cada uno de los archivos .zip
* Los descomprimiremos
* Los agruparemos en un único geodataframe (*gdf*)
* Lo exportareos a un archivo gpkg en la ruta de ../data/interim/ para un uso posterior. Este poligono será nuestro **ambito de estudio** (la cobertura total de las imágenes PNOA IRG)

In [3]:
# Ruta relativa a carpeta data/raw/raster/PNOA_IRG/
ruta_irg = '../data/raw/raster/PNOA_IRG/'

# Listar todos los archivos ZIP en la carpeta que contienen los shape con los marcos de las imágenes
zip_files = glob.glob(ruta_irg + '*.zip')

In [4]:
# Lista para almacenar cada GeoDataFrame. Crearemos un gdf para cada marco
gdf_list = []

for path in zip_files:
    # Leer el shapefile contenido en en cada ZIP
    gdf = gpd.read_file(f'zip://{path}')
    gdf_list.append(gdf)

In [5]:
zip_files

['../data/raw/raster/PNOA_IRG/PNOA_MA_IRG_OF_ETRS89_HU30_h25_0963_4.zip',
 '../data/raw/raster/PNOA_IRG/PNOA_MA_IRG_OF_ETRS89_HU30_h25_0963_1.zip',
 '../data/raw/raster/PNOA_IRG/PNOA_MA_IRG_OF_ETRS89_HU30_h25_0963_2.zip',
 '../data/raw/raster/PNOA_IRG/PNOA_MA_IRG_OF_ETRS89_HU30_h25_0963_3.zip']

In [6]:
# Unir todos los GeoDataFrames en uno solo
gdf_marcos_irg = gpd.GeoDataFrame(pd.concat(gdf_list, ignore_index=True))

# Guarda los valores ordenados en un archivo CSV
gdf_marcos_irg['HMTN25'].sort_values(ascending=True).to_csv('../data/interim/HMTN25.csv', index=False, header=False)

print('Número de filas:', gdf_marcos_irg.shape[0]) 
print('Número de columnas:', gdf_marcos_irg.shape[1])

gdf_marcos_irg.head()

Número de filas: 4
Número de columnas: 5


,HMTN25,RESOLUCION,FECHA,ESTADO,geometry
0,0963_4,0.25,2022-07,Definitivo,"POLYGON Z ((262528 4153248 0, 262370 4153248 0..."
1,0963_1,0.25,2022-07,Definitivo,"POLYGON Z ((248068 4162928 0, 247920 4162928 0..."
2,0963_2,0.25,2022-07,Definitivo,"POLYGON Z ((262808 4162498 0, 262640 4162498 0..."
3,0963_3,0.25,2022-07,Definitivo,"POLYGON Z ((247768 4153678 0, 247640 4153678 0..."


La información que tenemos para cada **marco** de cada imágen es la siguiente:
* **HMTNN25**: El numero de la hoja MTN25 que cubre. La imágen cubre un poco más que el la cuadrícula de la hoja MTN25 y tiene una pequeña superposición con las hojas adyacentes
* **RESOLUCION**: el tamaño del pixel en metros
* **FECHA**: Fecha de la captura de la imágen aérea (ortofoto)
* **ESTADO**: El estado de edición de la imágen aérea (ortofoto)

Esta información nos puede ser útil más adelante

Comprobamos ahora cuantos valores únicos tenemos en cada una de las columnas de datos para asegurarnos que todas tengan la misma RESOLUCIÓN y FECHA de captura

In [7]:
print('Valores únicos en ["HMTN25"]:', len(gdf_marcos_irg['HMTN25'].unique()))
print('Valores únicos en ["RESOLUCION"]:', len(gdf_marcos_irg['RESOLUCION'].unique()))
print('Valores únicos en ["FECHA"]:', len(gdf_marcos_irg['FECHA'].unique()))
print('Valores únicos en ["ESTADO"]:', len(gdf_marcos_irg['ESTADO'].unique()))

Valores únicos en ["HMTN25"]: 4
Valores únicos en ["RESOLUCION"]: 1
Valores únicos en ["FECHA"]: 1
Valores únicos en ["ESTADO"]: 1


Unimos los marcos (dissolve) de todas las imágenes para obtenr el **ámbito de estudio**. Esto se corresponderá con la superficie total cubierta por el conjunto de todas las imágenes aéreas

In [8]:
# Hacer un dissolve para combinar todos los polígonos en una sola geometría
# y tener un marco único que utilizaremos como ámbito del estudio
dissolve_marcos_irg = gdf_marcos_irg.dissolve()

# Eliminamos todos los campos excepto geometry (al disolver las geometrias el resto de campos ya no tienen sentido)
# Esté dissolve nos servirá como ámbito de estudio. Será el área que cubren todas las imágenes aéreas
dissolve_marcos_irg = dissolve_marcos_irg['geometry']

Guardamos el gdf de todos los marcos individuales y el de la únión de todos los marcos (**ámbito de estudio**) en archivos .gpkg

In [9]:
# Guardar las capas en GeoPackage (.gpkg) para su uso posterior
ruta_destino = '../data/interim/'
gdf_marcos_irg.to_file(os.path.join(ruta_destino, 'pnoa_irg_marcos.gpkg'), driver='GPKG')
dissolve_marcos_irg.to_file(os.path.join(ruta_destino, 'pnoa_irg_ambito.gpkg'), driver='GPKG')

## Imágenes PNOA IRG
**Web**: https://centrodedescargas.cnig.es/CentroDescargas/ortofoto-pnoa-falso-color-infrarrojo

Las ortofotos **PNOA en falso color infrarrojo (IRG)**, disponibles en el Centro de Descargas del CNIG, son mosaicos de imágenes aéreas que combinan las bandas infrarroja (4), roja (1) y verde (2). Estas imágenes tienen una resolución de píxel de 0,25 metros y se distribuyen en formato COG (Cloud Optimized GeoTIFF). Por comodidad los tenemos descargados junto con las imágenes en la ruta ../data/raw/raster/PNOA_IRG/

La principal ventaja de las imágenes en falso color infrarrojo frente a las convencionales RGB radica en su capacidad para resaltar diferencias en la vegetación y cuerpos de agua. El infrarrojo cercano es altamente reflectante en áreas con vegetación, mientras que el agua absorbe gran parte de esta radiación, apareciendo más oscura. Esto facilita la identificación y análisis de balsas de agua y otros cuerpos hídricos, mejorando la detección y monitoreo en comparación con las imágenes RGB tradicionales

* Comprobaremos que hay el mismo número de imagenges que de archivos zip con lso marcos de las imágenes
* Crearemos la tabla de correspondencia entre cada imágen y su marco

In [10]:
# Ruta relativa a carpeta data/raw/raster/PNOA_IRG/
ruta_irg = '../data/raw/raster/PNOA_IRG/'

# Listar todos los archivos TIF en la carpeta que contienen las imágenes PNOA IRG
tif_files = glob.glob(ruta_irg + '*.tif')

Comprobamos que todos los nombres de archivos de la carpeta tienen su correspondientes archivos zip (marco) y tif (imagen)

In [11]:
# Obtener nombres de los archivos sin extensiones
tif_names = {os.path.splitext(os.path.basename(f))[0] for f in tif_files}
zip_names = {os.path.splitext(os.path.basename(f))[0] for f in zip_files}

# Identificar archivos faltantes
missing_in_zip = tif_names - zip_names
missing_in_tif = zip_names - tif_names

# Imprimir resultados
if missing_in_zip:
    for name in missing_in_zip:
        print(f"Falta el archivo zip de {name}")
if missing_in_tif:
    for name in missing_in_tif:
        print(f"Falta el archivo tif de {name}")
if not missing_in_zip and not missing_in_tif:
    print("Todos los archivos tienen su correspondencia.")

Todos los archivos tienen su correspondencia.


Ahora crearemos un nuevo campo en gdf_marcos_irg ['RUTA_TIF'], con al ruta relativa de su archivo tif correspodiente y lo guardaremos en archivo pnoa_irg_marcos.gpkg sobreescribiendo el archivo anterior en la ruta ../data/interim/

In [12]:
# Crear un diccionario para mapear los valores de HMTN25 a las rutas de los archivos TIF
ruta_tif_mapping = {
    os.path.splitext(os.path.basename(tif))[0].split('_')[-2] + '_' + os.path.splitext(os.path.basename(tif))[0].split('_')[-1]: tif
    for tif in tif_files
}

# Añadir el campo RUTA_TIF al GeoDataFrame
gdf_marcos_irg['RUTA_TIF'] = gdf_marcos_irg['HMTN25'].map(ruta_tif_mapping)

# Reorganizar las columnas para que geometry siga siendo el último campo
cols = [col for col in gdf_marcos_irg.columns if col != 'geometry'] + ['geometry']
gdf_marcos_irg = gdf_marcos_irg[cols]

# Sobrescribimos el archivo anterior pnoa_irg_marcos.gpkg
gdf_marcos_irg.to_file(os.path.join(ruta_destino, 'pnoa_irg_marcos.gpkg'), driver='GPKG')

Al ser tan largos los nombres de las rutas, el gdf no los imprime completos. Así que para verificar que ha hecho la correspondencia correctamente generamos una muestra aleatoria de 5 elementos de gdf_marcos_irg e imprimimos los resultados

In [13]:
# Muestra aleatoria de 5 elementos
#muestra = gdf_marcos_irg.sample(n=5, random_state=42)

# Imprimimos los elementos de la muestra
#for index, row in muestra.iterrows():
    #print(f"HMTN25: {row['HMTN25']}")
    #print(f"RUTA_TIF: {row['RUTA_TIF']}")
    #print("-" * 80)

## Cartografía BTN del Centro de descargas del CNIG
**Web**: https://centrodedescargas.cnig.es/CentroDescargas/btn

**Especificaciones BTN (pdf)**: https://www.ign.es/resources/docs/IGNCnig/BTN/ESPBTN.pdf


La web del Centro de descargas del CNIG, ofrece los archivos de la ***Base Topográfica Nacional*** (**BTN**), una base de datos tridimensional multiescala con coberturas de 1:2.000 a 1:25.000 para España.  por hojas Los datos están disponibles por hojas MTN50 o por provincias enteras. Hemos descargado los archivos .zip de las provincias dentro de nuestro ámbito de estudio que contienen los archivos con las todas las geometrías vectoriales de la cartografía BTN en arcivos .shp organizados por categorías en el docuemnto pdf de las Especificaciones se informa del contenido de cada archivo y el significado de los código de sus variable. 

* Recorreremos cada uno de los archivos por provincias .zip
* Los descomprimiremos
* Selecionaremos los archivos shape de interes para nuestro proyecto (superficies con agua)
  * **BTN00325S_EMBALSE.shp**: Obra hidráulica que genera una masa de agua acumulada artificialmente en el transcurso de un cauce mediante una presa, para su aprovechamiento en regadíos, producción de energía, abastecimiento, etc
  * **BTN0316S_LAGUNA.shp**: Masa de agua acumulada en una depresión natural del terreno.
  * **BTN0328S_ALM_AGU.shp**: Elemento artificial construido con objeto de almacenar agua para fines diversos (Piscina, Balsa/Alberca/Estanque, Vaso de Salina).
  * **BTN0570S_CON_HID_S.shp**: Recinto en el que se ubican infraestructuras destinadas al tratamiento de aguas con objeto de acondicionarlas para conseguir un fin determinado (ETAP/EDAR/DESALINIZADORAS).
  * **BTN0302S_RIO.shp**: Corriente de agua natural más o menos caudalosa que desemboca en otra, en un lago o en el mar.
  * **BTN0302L_RIO.shp**: Corriente de agua natural más o menos caudalosa que desemboca en otra, en un lago o en el mar.
  * **BTN0305S_CAU_ART.shp**: Obra realizada para conducir una corriente de agua para distintos usos.
  * **0576S_DEP_GEN.shp**: Construcción cerrada para almacenar productos como hidrocarburos, aceites u otros (excepto el agua).
  * **BTN0334P_SURGEN.shp**: Lugar por donde surge o brota el agua. Una de las ategorías es la de 'Pozo' y quizas nos sirva, haciendo una seleción manual para identificar alguna morfología concreta. Al ser entidades puntuales no muy precisas se necesitará una supervisión manual, en caso de utilizarse.
  * **BTN0540S_EXP_MIN_S.shp**: Lugar destinado a la extracción y aprovechamiento de algún tipo de mineral o recurso.
  * **BTN0573S_ALM_RES.shp**: Lugar destinado al tratamiento, acopio o almacenaje de basura, residuos, materiales de desecho o escorias
de diferente tipo.
  * **BTN0564S_INS_DEP.shp**: Recinto en el que se desarrollan actividades deportivas, que puede albergar en su interior pistas deportivas u
otras instalaciones.
  * **BTN0561S_ZON_VER.shp**: Terreno ajardinado que se destina a parque o arbolado. También se utiliza para representar medianas
ajardinadas y el interior de los campos de golf.
   

* Los agruparemos en geodataframe (*gdf*) únicos por categorias.
* Los exportaremos a archivos gpkg en la ruta de ../data/interim/ para un uso posterior

In [14]:
# Directorio que contiene los archivos ZIP
directory = r'../data/raw/vector/cnig/btn/'

# Obtener la única geometría de gdf_marcos_irg para los filtros espaciales
ambito = dissolve_marcos_irg.union_all()

In [15]:
# Diccionario con los nombres de shapefiles y sus archivos de salida
shapefile_to_output = {
    "BTN0325S_EMBALSE.shp": "btn_embalse.gpkg",     # Polígonos de Embalses
    "BTN0316S_LAGUNA.shp": "btn_laguna.gpkg",       # Polígonos de Lagunas
    "BTN0328S_ALM_AGU.shp": "btn_alm_agu.gpkg",     # Polígonos de Almacenamiento de Agua (Piscinas, Balsas)
    "BTN0334P_SURGEN.shp": "btn_surgen.gpkg",       # Puntos de Surgencias (Fuente Manatial, Pozos)
    "BTN0570S_CON_HID_S.shp": "btn_con_hid_s.gpkg", # Polígonos Construciones Hidrográficas (ETAP, EDAR, Desalinizadoras)
    "BTN0302S_RIO.shp": "btn_rio_s.gpkg",           # Polígonos con tramos de ríos anchos
    "BTN0302L_RIO.shp": "btn_rio_l.gpkg",           # Líneas con tramos de ríos
    "BTN0305S_CAU_ART.shp": "btn_cau_art.gpkg",   # Polígonos con tramos de canales de agua
    "BTN0576S_DEP_GEN.shp": "btn_dep_gen.gpkg",   # Polígonos con localización de depósitos
    "BTN0540S_EXP_MIN_S.shp": "btn_exp_min_s.gpkg", # Polígonos con explotaciones mineras
    "BTN0573S_ALM_RES.shp": "btn_alm_res.gpkg",   # Polígonos con almacen de residuos
    "BTN0564S_INS_DEP.shp": "btn_ins_dep.gpkg",   # Polígonos con instalaciones deportivas
    "BTN0561S_ZON_VER.shp": "btn_zon_ver.gpkg"    # Polígonos con zonas verdes
}

In [16]:
# Función para extraer el shapefile específico de un zip y devolverlo como GeoDataFrame.
# Se utiliza el nombre base (sin extensión) para extraer todos los archivos asociados.
def extract_shapefile_from_zip(zip_path, shapefile_with_ext):
    base_name = os.path.splitext(shapefile_with_ext)[0]
    try:
        # Crear un directorio temporal para trabajar con los archivos
        with tempfile.TemporaryDirectory() as temp_dir:
            with zipfile.ZipFile(zip_path, 'r') as zip_ref:
                # Extraer archivos que comiencen con el nombre base
                shapefile_files = [f for f in zip_ref.namelist() if f.startswith(base_name)]
                zip_ref.extractall(path=temp_dir, members=shapefile_files)
                # Ruta completa al archivo .shp dentro del directorio temporal
                shapefile_path = os.path.join(temp_dir, shapefile_with_ext)
                # Cargar el shapefile como GeoDataFrame
                gdf = gpd.read_file(shapefile_path)
            return gdf
    except Exception as e:
        print(f"Error al procesar {shapefile_with_ext} en {os.path.basename(zip_path)}: {e}")
        return None

In [17]:
# Procesar cada shapefile definido en el diccionario
for shapefile_with_ext, output_filename in shapefile_to_output.items():
    gdfs = []  # Lista para almacenar los GeoDataFrames obtenidos para este shapefile
    for filename in os.listdir(directory):
        if filename.endswith(".zip"):
            zip_path = os.path.join(directory, filename)
            gdf = extract_shapefile_from_zip(zip_path, shapefile_with_ext)
            if gdf is not None:
                gdfs.append(gdf)
    if gdfs:
        # Fusionar los GeoDataFrames para esta categoría
        merged_gdf = gpd.GeoDataFrame(pd.concat(gdfs, ignore_index=True))
        # Filtrar las entidades que estén completamente contenidas en el ámbito
        filtered_gdf = merged_gdf[merged_gdf.geometry.within(ambito)]
        # Ruta de salida y guardado
        output_path = os.path.join('../data/interim/', output_filename)
        filtered_gdf.to_file(output_path, driver='GPKG')
        print(f"Guardado: {output_path}")
    else:
        print(f"No se encontraron datos válidos para {shapefile_with_ext}.")

Guardado: ../data/interim/btn_embalse.gpkg
Guardado: ../data/interim/btn_laguna.gpkg
Guardado: ../data/interim/btn_alm_agu.gpkg
Guardado: ../data/interim/btn_surgen.gpkg
Guardado: ../data/interim/btn_con_hid_s.gpkg
Guardado: ../data/interim/btn_rio_s.gpkg
Guardado: ../data/interim/btn_rio_l.gpkg
Guardado: ../data/interim/btn_cau_art.gpkg
Guardado: ../data/interim/btn_dep_gen.gpkg
Guardado: ../data/interim/btn_exp_min_s.gpkg
Guardado: ../data/interim/btn_alm_res.gpkg
Guardado: ../data/interim/btn_ins_dep.gpkg
Guardado: ../data/interim/btn_zon_ver.gpkg


**NOTA:** *Al utilizar los archivos gpkg generados en el apartado anterior, en procesos posteriores, detectamos que el campo numerico ID se ha guardado como Float por lo que vamos a crear una funcion para convertir este campo en Integer*

In [18]:
def convertir_id_a_integer(lista_gpkg, directorio):
    """
    Abre una lista de archivos GPKG, cambia el campo 'ID' a integer y guarda los cambios.
    """
    for nombre_gpkg in lista_gpkg:
        # Construir la ruta completa al archivo
        ruta_gpkg = os.path.join(directorio, nombre_gpkg)

        # Verificar si el archivo existe
        if not os.path.exists(ruta_gpkg):
            print(f"El archivo {ruta_gpkg} no existe. Se omitirá.")
            continue

        # Cargar el archivo GPKG en un GeoDataFrame
        gdf = gpd.read_file(ruta_gpkg)

        # Verificar si la columna 'ID' existe
        if 'ID' not in gdf.columns:
            print(f"El archivo {ruta_gpkg} no tiene una columna 'ID'. Se omitirá.")
            continue

        # Convertir la columna 'ID' a integer
        gdf['ID'] = gdf['ID'].astype(int)

        # Guardar el GeoDataFrame modificado en el mismo archivo GPKG
        gdf.to_file(ruta_gpkg, driver='GPKG')
        print(f"El archivo {ruta_gpkg} ha sido guardado con 'ID' como integer.")

# Ejemplo de uso
directorio = '../data/interim'  # Cambia esto por la ruta correcta
lista_gpkg = shapefile_to_output.values() # Values del diccionario qeu creamos antes

convertir_id_a_integer(lista_gpkg, directorio) 

El archivo ../data/interim/btn_embalse.gpkg ha sido guardado con 'ID' como integer.
El archivo ../data/interim/btn_laguna.gpkg ha sido guardado con 'ID' como integer.
El archivo ../data/interim/btn_alm_agu.gpkg ha sido guardado con 'ID' como integer.
El archivo ../data/interim/btn_surgen.gpkg ha sido guardado con 'ID' como integer.
El archivo ../data/interim/btn_con_hid_s.gpkg ha sido guardado con 'ID' como integer.
El archivo ../data/interim/btn_rio_s.gpkg ha sido guardado con 'ID' como integer.
El archivo ../data/interim/btn_rio_l.gpkg ha sido guardado con 'ID' como integer.
El archivo ../data/interim/btn_cau_art.gpkg ha sido guardado con 'ID' como integer.
El archivo ../data/interim/btn_dep_gen.gpkg ha sido guardado con 'ID' como integer.
El archivo ../data/interim/btn_exp_min_s.gpkg ha sido guardado con 'ID' como integer.
El archivo ../data/interim/btn_alm_res.gpkg ha sido guardado con 'ID' como integer.
El archivo ../data/interim/btn_ins_dep.gpkg ha sido guardado con 'ID' como int

## Filtrado de rios línea

Filtraremos solo los ríos importante utilizando el campo JERAR_0302. 

In [19]:
# Cargar el archivo en un GeoDataFrame
gdf_rio_l = gpd.read_file("../data/interim/btn_rio_l.gpkg")

# Filtrar por el campo JERAR_0302 que contenga alguno de los valores ('01', '02', '03')
valores_filtro_rio_l = ('01', '02', '03')
gdf_rio_l = gdf_rio_l[gdf_rio_l['JERAR_0302'].isin(valores_filtro_rio_l)]

# Guardar el GeoDataFrame filtrado en el mismo archivo (sobrescribiendo)
gdf_rio_l.to_file("../data/interim/btn_rio_l.gpkg", driver="GPKG")

## Filtrado de balsas

Ahora selecionaremos mediante varios filtros, los poligonos más representativos para el objetivo de nuestro proyecto,  utilizando la mejor información de la que disponemos. 

### Filtrado de geometrias BTN según sus características

Consultando la documentación de los atributos de la cartografía BTN podemos diferenciar elementos en las capas de **ALM_AGUA** y **LAGUNA**: 

**LAGUNA**
Vamos a generar un archivo independientes para:
* **[RÉGIMEN]** = 'PERMANENTE' **<span style="color:green;">&</span>** **[TIPO]** = 'AGUA DULCE'

<img src="./img/0316_Laguna.png" alt="0316_Laguna" width="50%">

**ALM_AGUA**
Vamos a generar archivos independientes para:
* **[TIPO]** = 'PISCINA' **<span style="color:green;">&</span>** **[ESTADO]** = 'EN USO'
* **[TIPO]** = 'BALSA/ALBERCA/ESTANQUE' **<span style="color:green;">&</span>** **[ESTADO]** = 'EN USO'

<img src="./img/0328_Almacenamiento_de_agua.png" alt="0328_Almacenamiento_de_agua" width="50%">

In [20]:
# Cargar los archivos GeoPackage en gdf
btn_laguna = gpd.read_file("../data/interim/btn_laguna.gpkg")
btn_alm_agu = gpd.read_file("../data/interim/btn_alm_agu.gpkg")

In [21]:
# Filtramos las lagunas permanentes de agua dulce
btn_laguna_perm_dulce = btn_laguna[(btn_laguna['TIPO_0316'] == '01') & (btn_laguna['REGIM_0316'] == '01')]

# Separamos los elementos de alm_agu por tipo (piscina y balsas) y filtramos por 'en uso'
btn_piscina_en_uso = btn_alm_agu[(btn_alm_agu['TIPO_0328'] == '01') & (btn_alm_agu['ESTAD_0328'] == '01')]
btn_balsa_en_uso = btn_alm_agu[(btn_alm_agu['TIPO_0328'] == '04') & (btn_alm_agu['ESTAD_0328'] == '01')]

In [22]:
# Eliminamos las balsas/albercas/estanques en parcelas Urbanas. Pueden llevar a confusión en la detección
btn_balsa_en_uso = btn_balsa_en_uso[btn_balsa_en_uso['TIPO_PARCE'] != 'U']

In [23]:
# Nos quedamos solo con los campos ['ID', 'geometry']
btn_laguna_perm_dulce = btn_laguna_perm_dulce[['ID', 'geometry']]
btn_piscina_en_uso = btn_piscina_en_uso[['ID', 'geometry']]
btn_balsa_en_uso = btn_balsa_en_uso[['ID', 'geometry']]

### Union de balsas solapadas o adyacentes

En el caso de las balsas de agua, si una de ellas se situa entre 2 o más parcelas catastrales puede estar dividida en 2 o más polígonos adyacentes. Detectaremos si existe solapamiento o intersección de lineas de las geometrias y las uniremos en un único polígonos.

1. Identificar polígonos que comparten un lado común.
2. Fusionar esos polígonos en una sola geometría.

In [24]:
# Mostrar tamaño del gdf original
btn_balsa_en_uso.shape

(437, 2)

In [25]:
import geopandas as gpd
from shapely.ops import unary_union
from rtree import index

# Corregir geometrías inválidas
btn_balsa_en_uso['geometry'] = btn_balsa_en_uso.geometry.buffer(0)

# Usamos R-tree para búsqueda espacial eficiente
idx_rtree = index.Index()
for i, geom in enumerate(btn_balsa_en_uso.geometry):
    idx_rtree.insert(i, geom.bounds)

# Inicializamos variables a utilizar
grupos = []
procesados = set()

for i in range(len(btn_balsa_en_uso)):
    if i in procesados:
        continue

    # Encontrar todos los polígonos adyacentes o intersectados usando R-tree
    por_procesar = {i}
    grupo = []

    while por_procesar:
        current = por_procesar.pop()
        if current in procesados:
            continue

        procesados.add(current)
        grupo.append(current)

        # Buscar vecinos potenciales usando R-tree
        geom = btn_balsa_en_uso.iloc[current].geometry
        posibles_vecinos = list(idx_rtree.intersection(geom.bounds))

        for vecino in posibles_vecinos:
            if vecino not in procesados:
                vecino_geom = btn_balsa_en_uso.iloc[vecino].geometry
                if geom.touches(vecino_geom) or geom.intersects(vecino_geom):
                    por_procesar.add(vecino)

    if grupo:
        grupos.append(grupo)

# Fusionar geometrías en cada grupo y mantener atributos del primer polígono
geometrias_fusionadas = []
for grupo in grupos:
    primer_poligono = btn_balsa_en_uso.iloc[grupo[0]]
    geometria_fusionada = unary_union(btn_balsa_en_uso.iloc[grupo].geometry)
    geometrias_fusionadas.append(geometria_fusionada)

# Crear nuevo GeoDataFrame con los atributos del primer polígono en cada grupo
btn_balsa_en_uso_fusionado = gpd.GeoDataFrame(
    btn_balsa_en_uso.iloc[[grupo[0] for grupo in grupos]].drop(columns='geometry'),
    geometry=geometrias_fusionadas,
    crs=btn_balsa_en_uso.crs
)

# Mostrar tamaño del gdf con polígonos fusionados
print(btn_balsa_en_uso_fusionado.shape)



(436, 2)


### Filtrado espacial de balsas

Ahora filtraremos los poligonos orignales de las categoriás BALSA/ALBERCA/ESTANQUE por otra capa de de balsas diponible a traves del área e descargas del Portal Ambiental de la Junta de Andalucia: REDIAM (Red de Información Ambiental de Andalucía), en la web: 
* https://portalrediam.cica.es/descargas?path=%2F10_SISTEMAS_PRODUCTIVOS%2F06_RECURSOS_HIDROLOGICOS%2FBalsas_2020_SIPNA20_2024_02%2FInfGeografica%2FInfVectorial%2FGeoparquet


* Este archivo **balsas2020_pub2024.parquet** contiene la "*Cartografía de Balsas año 2020, extraídas del Sistema de Información sobre el Patrimonio Natural de Andalucía (SIPNA)*", con las categorias:
    * **Depósito de alpechín**: Instalación destinada a almacenar el residuo líquido generado en las almazaras durante la extracción del aceite de oliva. Este residuo, conocido como alpechín, contiene agua vegetal, polifenoles y materia orgánica.
    * **Balsa de riego o ganadera**: Estructura diseñada para almacenar agua destinada al riego agrícola o para uso ganadero. Estas balsas suelen construirse con materiales impermeables y permiten garantizar un suministro hídrico constante.
    * **Balsa industrial o minera**: Instalación utilizada para almacenar residuos líquidos y sólidos generados en procesos industriales o mineros, como relaves o lodos. Estas balsas están diseñadas para contener materiales tóxicos y prevenir la contaminación del suelo y las aguas circundantes.
    * **Lámina de agua artificial**: Superficie de agua creada artificialmente, generalmente mediante excavación o construcción, para fines recreativos, paisajísticos o como parte de sistemas de gestión hídrica. Estas láminas pueden incluir embalses pequeños o estanques diseñados para diversos usos humanos.

* La cartografía del SIPNA tiene una escala de detalle 1:10.000, lo que permite un análisis detallado de los elementos geográficos y medioambientales, como ocupación del suelo, vegetación y hábitats.

-------------------------------------------
**NOTA:** Utilizaremos las geometrías (polígonos) de la capa de la BTN pero selecionando solo las que aparezcan en la capa de la REDIAM *Balsa de riego o ganadera* por las siguiente razones:
1. La capa **BTN0328S_ALM_AGU.shp** tiene geometrías más limpias, más definidas y de mayor detalle y menor escala, por lo tanto son mejores que las de **REDIAM** para delimitar lso permitros de las balsas
2. La categoría *Balsa/Alberca/Estanque* de la capa **BTN0328S_ALM_AGU.shp** en un cajón de sastre en la que además de las balsas hay Albercas y Estanques. Utilizaremos la categoría *Balsa de riego o ganadera* para filtrar espacialmente solo las Balsas.
3. Finalmente filtraremos por área mínima las Balsas más pequeñas que no tienen interes para nuestro proyecto e introducen mucho ruido para el entrenamiento del modelo.

In [26]:
# Cargamos el geoparquet en una gdf
gdf_balsas_rediam = gpd.read_parquet("../data/raw/vector/rediam/balsas2020_pub2024.parquet")

# Reproyectamos al crs EPSG:25830
gdf_balsas_rediam = gdf_balsas_rediam.to_crs(epsg=25830)

# Filtramos las geometrías contenidas totalmente dentro del 'ambito'
# ambito ya lo utilizamos anteriormente en este notebook para filtrar las capas btn
gdf_balsas_rediam = gdf_balsas_rediam[gdf_balsas_rediam.geometry.within(ambito)]

# Guardamos en GPKG todas las Balsas REDIAM contenidas en nuestro ámbito (sin filtrar por Riego o Ganadera)
gdf_balsas_rediam.to_file("../data/interim/rediam_balsas_todas.gpkg", driver='GPKG')

# Filtramos por el valor en el campo os_des_n4 = Balsa de riego o ganadera
gdf_balsas_rediam = gdf_balsas_rediam[gdf_balsas_rediam['os_des_n4'] == 'Balsa de riego o ganadera']

# Guardamos en GPKG los poligonos de las balsas REDIAM de uso = Riego o Ganadero
gdf_balsas_rediam['geometry'].to_file("../data/interim/rediam_balsas.gpkg", driver='GPKG')

#gdf_balsas_rediam.head()

In [27]:
# Selección espacial de las balsas de la BTN que intersectan con las balsas de REDAIM usando sjoin
btn_balsa_en_uso = gpd.sjoin(btn_balsa_en_uso, gdf_balsas_rediam, how="inner", 
                             predicate="intersects")[btn_balsa_en_uso.columns]

### Filtrado por superficie mínima

* Filtramos las balsas por una superfície mńimo, que despues de un analisis visual establacemos en **200 m$^2$**. 
    * Las balsas más pequeñas suelen pertenecer a la categoria de albercas, ubicadas al lado de naves/granjas
    * En muchos casos estos almacenes de agua pequeños estan vacios o abandonadas
    * Introducen mucho ruido para la clasificación
    * En el Inventario de Balsas de Andalucía del 2006 establacen una superficie mínima de 700 m2 (Excepto en Almería por las balsas de lso invernaderos que lo establcen en 150-200 m$^2$)


In [28]:
# Calcular el área en metros cuadrados
btn_balsa_en_uso['area_m2'] = btn_balsa_en_uso.geometry.area

In [29]:
# Filtramos por el las balsas mayores o igual a 200 m2
btn_balsa_en_uso = btn_balsa_en_uso[btn_balsa_en_uso['area_m2'] >= 200]

In [30]:
# Guardamos en GPKG
btn_laguna_perm_dulce.to_file('../data/interim/btn_laguna_perm_dulce.gpkg', driver='GPKG')
btn_piscina_en_uso.to_file('../data/interim/btn_piscina_en_uso.gpkg', driver='GPKG')
btn_balsa_en_uso.to_file('../data/interim/btn_balsa_en_uso.gpkg', driver='GPKG')

In [31]:
print('LAGUNAS Totales (Sin filtros):', btn_laguna.shape[0])
print('LAGUNAS Permanentes de Agua Dulce:', btn_laguna_perm_dulce.shape[0])

LAGUNAS Totales (Sin filtros): 33
LAGUNAS Permanentes de Agua Dulce: 14


In [32]:
print('ALM_AGU Totales (Sin filtros):', btn_alm_agu.shape[0])
print('ALM_AGU Piscina en Uso:', btn_piscina_en_uso.shape[0])
print('ALM_AGU Balsas en Uso:', btn_balsa_en_uso.shape[0])

ALM_AGU Totales (Sin filtros): 2470
ALM_AGU Piscina en Uso: 2013
ALM_AGU Balsas en Uso: 207


### Filtrado por inspección visual
* Durante las pruebas realizadas para este proyecto, se ha creado un lisdato de ID de balsas que se consideran inapropiadas para el entrenamiento por varias razones:
    * Porque ya no existen
    * Porque aún no existian cuado se obtuvieron las imaǵenes
    * Porque no se consideran balss "tipo"
    * Porque están abandonadas y/o cubiertas de vegetación
* Para evitar ue el modelo de detección aprenda patrónes erroneas se recopilarań los IDs y se eliminarán en esta parte del flujo de trabajo

In [33]:
# Listado de IDs de balsas 'malas'
lista_ids_quitar = [246797711, 246797852, 246797995, 246804163, 246804696, 
                    246804731, 246804732, 246805836, 246805839, 246806121,  
                    246806325, 246806637, 246807279, 246807280, 246807288,  
                    246808291, 246809315, 246809733, 246809854, 246810519,  
                    246810794, 246811011, 246811258, 246811516, 246843485,  
                    246847031,  246864775]

In [34]:
# Imprimir el numero de balsas antes del filtro
print("Nº balsas antes del filtro", len(btn_balsa_en_uso))

# Utilizar .loc para seleccionar las filas donde el 'ID' NO está en la lista a quitar.
btn_balsa_en_uso = btn_balsa_en_uso.loc[~btn_balsa_en_uso['ID'].isin(lista_ids_quitar)]

# Imprimir el número de balsas después del filtro
print("Nº balsas despés del filtro", len(btn_balsa_en_uso))

Nº balsas antes del filtro 207
Nº balsas despés del filtro 202


In [35]:
# Eliminar filas duplicadas (considerando todas las columnas)
btn_balsa_en_uso = btn_balsa_en_uso.drop_duplicates()

In [36]:
# Guardamos gdf en gpkg
btn_balsa_en_uso.to_file('../data/interim/btn_balsa_en_uso.gpkg', driver='GPKG')

## Asignara a las balsas la HMTN25 de la imagen PNOA a la que 'pertenecen'

Queremos que cada una de las balsas tenga asignado la referencia del nombre del marco de la imágen PNOA IRG que le ofrece mejor cobertura.

In [37]:
# Invocamos a los gdf creados en notebook anteriormente
balsas_hmtn25 = btn_balsa_en_uso.copy()
marcos_hmtn25 = gdf_marcos_irg

# Inicializar la columna HMTN25 con valores nulos
balsas_hmtn25['HMTN25'] = None

# Creamos un índice espacial para marcos mejora considerablemente el rendimiento
marcos_sindex = marcos_hmtn25.sindex

# Preprocesamos los bordes de los marcos
marcos_hmtn25['geometry_boundary'] = marcos_hmtn25.geometry.boundary

# Nos aseguramos que ambas gdf tengan el mismo CRS
if balsas_hmtn25.crs != marcos_hmtn25.crs:
    raise ValueError("Los CRS de balsas y marcos no coinciden. Reprojéctalos antes de continuar.")

In [38]:
# Procesar cada balsa
for idx, balsa in balsas_hmtn25.iterrows():
    # Usar índice espacial para encontrar marcos candidatos que intersectan
    posibles_marcos_idx = list(marcos_sindex.intersection(balsa.geometry.bounds))
    if not posibles_marcos_idx:
        continue
        
    marcos_candidatos = marcos_hmtn25.iloc[posibles_marcos_idx]
    
    # Filtrar marcos que contienen completamente la balsa
    marcos_completos = marcos_candidatos[marcos_candidatos.contains(balsa.geometry)]
    
    if not marcos_completos.empty:
        # Calcular distancias en una sola operación vectorizada
        distancias = marcos_completos.geometry_boundary.distance(balsa.geometry)
        
        # Encontrar el marco con la mayor distancia al borde
        idx_max_distancia = distancias.idxmax()
        
        # Asignar el código del marco seleccionado
        balsas_hmtn25.loc[idx, 'HMTN25'] = marcos_completos.loc[idx_max_distancia, 'HMTN25']

In [39]:
# Comprobamos que todos los registros tengan HMTN25 asignada
balsas_hmtn25.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Index: 202 entries, 0 to 2464
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   ID        202 non-null    int64   
 1   geometry  202 non-null    geometry
 2   area_m2   202 non-null    float64 
 3   HMTN25    202 non-null    object  
dtypes: float64(1), geometry(1), int64(1), object(1)
memory usage: 16.0+ KB


In [40]:
# Guardamos resultado sobreescribiendo el gpkg
balsas_hmtn25.to_file('../data/interim/btn_balsa_en_uso.gpkg', driver='GPKG')

# Visualizaciones

Crearemos unas visualizaciones rápidas de las geometrías que hemos obtenido con los pasos previos utilizando la librería de python **Folium** que facilita la visualización de datos geoespaciales en mapas interactivos basados en Leaflet.js. 

**Folium** pág. web: https://python-visualization.github.io/folium/latest/

In [41]:
# Reproyectamos los gdf a EPSG:4326 (latitud y longitud) para visualizarlas en Folium
dissolve_marcos_irg_4326 = dissolve_marcos_irg.to_crs(epsg=4326)
btn_piscina_en_uso_4326 = btn_piscina_en_uso.to_crs(epsg=4326)[['geometry']]
btn_balsa_en_uso_4326 = btn_balsa_en_uso.to_crs(epsg=4326)[['geometry']]
gdf_balsas_rediam_4326 = gdf_balsas_rediam.to_crs(epsg=4326)[['geometry']]

### Balsas y piscinas

Representamos en un mapa interactivo la distribución espacial de las **balsas** y **piscinas** dentro del ámbito de estudio.

In [42]:
# Crear mapa centrado en dissolve_marcos_irg
x, y = dissolve_marcos_irg_4326.geometry.centroid.x.mean(), dissolve_marcos_irg_4326.geometry.centroid.y.mean()
m = folium.Map(location=[y, x], zoom_start=10)


# Añadir WMS PNOA IRG
folium.WmsTileLayer(
    'https://www.ideandalucia.es/wms/ortofoto_2022?',
    layers='ortofotografia_2022_infrarrojo',
    fmt='image/jpeg',
    transparent=True,
    name='WMS Ortofoto IRG',
    control=True
).add_to(m)

# Añadir dissolve_marcos_irg_4326 con simbología (sin relleno, borde rojo grueso)
folium.GeoJson(
    dissolve_marcos_irg_4326,
    style_function=lambda x: {
        'fillColor': 'none',
        'color': 'green',
        'weight': 5
    },
    name="Ámbito"
).add_to(m)

# Añadir btn_piscina_en_uso_4326
folium.GeoJson(
    btn_piscina_en_uso_4326,
    style_function=lambda x: {
        'fillColor': 'cyan',
        'color': 'cyan',
        'weight': 2,
        'fillOpacity': 0.5
    },
    name="Piscinas"
).add_to(m)


# Añadir btn_balsa_en_uso_4326
folium.GeoJson(
    btn_balsa_en_uso_4326,
    style_function=lambda x: {
        'fillColor': 'blue',
        'color': 'blue',
        'weight': 2,
        'fillOpacity': 0.5
    },
    name="Balsas BTN"
).add_to(m)

# Añadir gdf_balsas_rediam_4326
folium.GeoJson(
    gdf_balsas_rediam_4326,
    style_function=lambda x: {
        'fillColor': 'yellow',
        'color': 'yellow',
        'weight': 2,
        'fillOpacity': 0.0
    },
    name="Balsas REDIAM"
).add_to(m)


# Agregar el control de capas
folium.LayerControl(collapsed=False).add_to(m)

# Crear leyenda HTML
legend_html = """
<div style="
    position: fixed; 
    bottom: 50px; 
    right: 50px; 
    width: 200px; 
    height: auto; 
    border: 2px solid grey; 
    z-index: 9999; 
    font-size: 14px;
    background-color: white;
    padding: 10px;
    ">
    <b>Leyenda</b> <br>
    <i style="border: 4px solid green; width: 20px; height: 10px; display: inline-block;"></i> Ámbito <br>
    <i style="border: 2px solid cyan; width: 20px; height: 10px; display: inline-block;"></i> Piscina <br>
    <i style="border: 2px solid blue; width: 20px; height: 10px; display: inline-block;"></i> Balsas BTN <br>
    <i style="border: 2px solid yellow; width: 20px; height: 10px; display: inline-block;"></i> Balsas REDIAM <br>
</div>
"""

# Crear título HTML
titulo_html = """
<div style="
    position: fixed; 
    top: 10px; 
    left: 50%; 
    transform: translateX(-50%); 
    z-index: 1000; 
    font-size: 20px; 
    font-weight: bold;
    text-align: center; 
    background-color: white; 
    border: 2px solid black; 
    padding: 10px; 
    border-radius: 5px;
    box-shadow: 3px 3px 5px rgba(0,0,0,0.3);">
    Mapa interactivo:<br> Balsas y Piscinas
</div>
"""

# Añadir leyenda al mapa
m.get_root().html.add_child(folium.Element(legend_html))

# Añadir título al mapa
m.get_root().html.add_child(folium.Element(titulo_html))

# Mostrar el mapa
m

/tmp/ipykernel_21985/2792415551.py:2: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  x, y = dissolve_marcos_irg_4326.geometry.centroid.x.mean(), dissolve_marcos_irg_4326.geometry.centroid.y.mean()
